## 缺失值处理

In [483]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import matplotlib as mpl
import scipy.stats as stats
import numpy as np
import datetime

In [484]:
train = pd.read_csv('地区温度预测挑战赛公开数据/train.csv')
test = pd.read_csv('地区温度预测挑战赛公开数据/test.csv')
submit = pd.read_csv('地区温度预测挑战赛公开数据/sample_submit.csv')

风速和风向两列缺失值太多了，删除

In [485]:
train.dropna(axis=0, subset=['气温'], inplace=True)
data_all = pd.concat([train, test])
data_all = data_all.drop(['平均每小时风速', '主要每小时风向'], axis=1)
data_all = data_all.fillna(method='ffill')

## 时间序列特征

In [486]:
data_all['时间'] = data_all['年'].map(str)+"/"+data_all['月'].map(str)+"/"+data_all['日'].map(str)
data_all['时间'] = pd.to_datetime(data_all['时间'])
data_all.drop(labels=['年', '月', '日'], axis=1)

,降水量,气温,湿球空气温度,露点空气温度,蒸气压,相对湿度,平均海平面压力,地区,站号,时间
0,0.0,1.4,0.8,-0.1,6.2,90.1,1016.3,0,0,2016-01-01 00:00:00
1,0.0,1.6,0.8,-0.7,6.0,85.0,1016.3,0,0,2016-01-01 01:00:00
2,0.0,-0.2,-0.5,-1.3,5.6,92.2,1016.4,0,0,2016-01-01 02:00:00
3,0.0,-0.5,-1.0,-1.3,5.7,95.0,1016.3,0,0,2016-01-01 03:00:00
4,0.0,-0.3,-0.5,-1.1,5.8,94.1,1016.2,0,0,2016-01-01 04:00:00
...,...,...,...,...,...,...,...,...,...,...
17851,0.0,6.9,10.6,10.4,12.5,96.0,1005.8,7,23,2021-12-31 19:00:00
17852,0.0,6.9,10.6,10.3,12.4,95.2,1004.9,7,23,2021-12-31 20:00:00
17853,0.2,6.9,11.0,10.9,13.1,96.0,1003.9,7,23,2021-12-31 21:00:00
17854,0.1,6.9,11.7,11.3,13.4,98.1,1002.7,7,23,2021-12-31 22:00:00


In [487]:
def get_feature1(df1):
    df1['时间'] = df1['年'].map(str)+"/"+df1['月'].map(str)+"/"+df1['日'].map(str)
    df1['时间'] = pd.to_datetime(df1['时间'])
    df1.drop(labels=['年', '月', '日'], axis=1, inplace=True)
    df1['日期'] = df1['时间'].dt.date 
    df1['day'] = df1['时间'].dt.day
    df1['hour_of_day'] = df1['时间'].dt.hour
#     df1['minute_of_day'] = df1['时间'].dt.minute
    df1['month_of_year'] = df1['时间'].dt.month
#     df1['time'] = df1['时间'].dt.time
    df1['quarter'] = df1['时间'].dt.quarter
    df1['day_of_year'] = df1['时间'].dt.dayofyear
#     df1['day_of_week'] = df1['时间'].dt.dayofweek #星期中的哪一天
    df1.drop(labels=['时间'], axis=1, inplace=True)

In [488]:
get_feature1(data_all)

In [489]:
data_all

,降水量,气温,湿球空气温度,露点空气温度,蒸气压,相对湿度,平均海平面压力,地区,站号,日期,day,hour_of_day,month_of_year,quarter,day_of_year
0,0.0,1.4,0.8,-0.1,6.2,90.1,1016.3,0,0,2016-01-01,1,0,1,1,1
1,0.0,1.6,0.8,-0.7,6.0,85.0,1016.3,0,0,2016-01-01,1,1,1,1,1
2,0.0,-0.2,-0.5,-1.3,5.6,92.2,1016.4,0,0,2016-01-01,1,2,1,1,1
3,0.0,-0.5,-1.0,-1.3,5.7,95.0,1016.3,0,0,2016-01-01,1,3,1,1,1
4,0.0,-0.3,-0.5,-1.1,5.8,94.1,1016.2,0,0,2016-01-01,1,4,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17851,0.0,6.9,10.6,10.4,12.5,96.0,1005.8,7,23,2021-12-31,31,19,12,4,365
17852,0.0,6.9,10.6,10.3,12.4,95.2,1004.9,7,23,2021-12-31,31,20,12,4,365
17853,0.2,6.9,11.0,10.9,13.1,96.0,1003.9,7,23,2021-12-31,31,21,12,4,365
17854,0.1,6.9,11.7,11.3,13.4,98.1,1002.7,7,23,2021-12-31,31,22,12,4,365


## 归一化

In [490]:
cols_numeric = list(data_all.columns[:6])
def scale_minmax(col):
    return (col - col.min()) / (col.max() - col.min())


data_all[cols_numeric] = data_all[cols_numeric].apply(scale_minmax, axis=0)
data_all[cols_numeric].describe()

,降水量,气温,湿球空气温度,露点空气温度,蒸气压,相对湿度
count,1.262365e+06,1.262365e+06,1.262365e+06,1.262365e+06,1.262365e+06,1.262365e+06
mean,3.722658e-03,4.572760e-01,7.952782e-01,5.906490e-01,3.469589e-01,7.883478e-01
std,1.402581e-02,1.209421e-01,5.969329e-02,1.209631e-01,1.276478e-01,1.482455e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,3.715013e-01,7.530529e-01,5.013405e-01,2.450593e-01,6.995012e-01
50%,0.000000e+00,4.580153e-01,7.978290e-01,5.951743e-01,3.359684e-01,8.229426e-01
75%,0.000000e+00,5.445293e-01,8.398915e-01,6.809651e-01,4.347826e-01,9.102244e-01
max,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [491]:
data_all

,降水量,气温,湿球空气温度,露点空气温度,蒸气压,相对湿度,平均海平面压力,地区,站号,日期,day,hour_of_day,month_of_year,quarter,day_of_year
0,0.000000,0.231552,0.685210,0.388740,0.166008,0.874065,1016.3,0,0,2016-01-01,1,0,1,1,1
1,0.000000,0.236641,0.685210,0.372654,0.158103,0.810474,1016.3,0,0,2016-01-01,1,1,1,1,1
2,0.000000,0.190840,0.667571,0.356568,0.142292,0.900249,1016.4,0,0,2016-01-01,1,2,1,1,1
3,0.000000,0.183206,0.660787,0.356568,0.146245,0.935162,1016.3,0,0,2016-01-01,1,3,1,1,1
4,0.000000,0.188295,0.667571,0.361930,0.150198,0.923940,1016.2,0,0,2016-01-01,1,4,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17851,0.000000,0.371501,0.818182,0.670241,0.415020,0.947631,1005.8,7,23,2021-12-31,31,19,12,4,365
17852,0.000000,0.371501,0.818182,0.667560,0.411067,0.937656,1004.9,7,23,2021-12-31,31,20,12,4,365
17853,0.005882,0.371501,0.823609,0.683646,0.438735,0.947631,1003.9,7,23,2021-12-31,31,21,12,4,365
17854,0.002941,0.371501,0.833107,0.694370,0.450593,0.973815,1002.7,7,23,2021-12-31,31,22,12,4,365


In [492]:
train_X = data_all[data_all['日期']<datetime.date(2021,12,1)]
test_X = data_all[data_all['日期']>=datetime.date(2021,12,1)]
train_label = train_X['气温']
train_X.drop(['日期', '气温'], axis=1, inplace=True)
test_X.drop(['日期', '气温'], axis=1, inplace=True)

D:\software\anaconda2022\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [ ]:
from sklearn.ensemble import RandomForestRegressor
train_data, val_data, train_target, val_target = train_test_split(train_X, train_label, test_size=0.2, random_state=0)
clf = RandomForestRegressor(random_state=0, n_estimators=200)
clf = clf.fit(train_data, train_target)
clf.score(val_data, val_target)

In [ ]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error 
train_data, val_data, train_target, val_target = train_test_split(train_X, train_label, test_size=0.2, random_state=0)
reg_lgb = lgb.LGBMRegressor(
    learning_rate=0.01,
    max_depth=-1,
    n_estimators=5000,
    boosting_type='gbdt',
    random_state=2019,
    objective='regression',
)

#训练模型
reg_lgb.fit(train_data, train_target)
score = mean_squared_error(val_target, reg_lgb.predict(val_data))
print('LightGBM socre ', score)
#LightGBM socre  0.156274722993209

In [ ]:
from sklearn.model_selection import KFold 
import lightgbm as lgbm 

lgb_params = {
      "objective": "mae", 
      "metric": "mae", 
      "boosting_type": "gbdt",
      'early_stopping_rounds': 150,
      'learning_rate': 0.01,  
      'colsample_bytree':0.95, 
}
# X_tr_val = df_train.copy()
# X_te     = df_test[features]
 
kf = KFold(n_splits=5)
lgb_models = []
X = train_data
y_pred = 0
for f,(train_index,valid_index) in enumerate(kf.split(X)):
    
    X_train,X_valid = X.iloc[train_index], X.iloc[valid_index]
    y_train,y_valid = train_label.iloc[train_index], train_label.iloc[valid_index]
    lgbm_train = lgbm.Dataset(X_train,y_train)  
    lgbm_valid = lgbm.Dataset(X_valid,y_valid)

    model_mae = lgbm.train(params=lgb_params, 
                  train_set=lgbm_train,
                  valid_sets=[lgbm_train, lgbm_valid],
                  num_boost_round=1000,   
                  verbose_eval=100,)
#                   categorical_feature = cate_features)
    y_pred = model_mae.predict(X_train) / 5.0
    lgb_models.append(model_mae) 